## **Priority Scheduling (PS) Code**

In [1]:
sample_processes=[ #Sample dataset with pid, arrival time, burst time, and priority (can add more PIDs or the values can be changed)
    {"pid": "P1", "arrival": 0, "burst": 10, "priority": 4},
    {"pid": "P2", "arrival": 1, "burst": 6, "priority": 3},
    {"pid": "P3", "arrival": 2, "burst": 11, "priority": 5},
    {"pid": "P4", "arrival": 3, "burst": 7, "priority": 4}
]

def print_schedule(results): #Function to print scheduling results in a table
    print(f"{'PID':<5} {'Start':<6} {'Finish':<6} {'Waiting':<7} {'Turnaround':<10}") #Print the table with fixed column widths
    for r in results: #Loop through each result entry (each process)
        print(f"{r['pid']:<5} {r['start']:<6} {r['finish']:<6} "
              f"{r['waiting']:<7} {r['turnaround']:<10}")

def priority_non_preemptive(sample_processes):
    time = 0 #Current simulation time
    results = [] #Stores scheduling results
    done = [] #Tracks completed processes

    while len(done) < len(sample_processes): #Loop until all processes are finished
        available = [p for p in sample_processes if p["arrival"] <= time and p not in done] #Get all processes that have arrived and are not completed

        if not available: # If nothing is ready to run, advance time until a process arrives
            time += 1
            continue

        p = min(available, key=lambda x: x["priority"]) #Select process with highest priority (lowest priority number)

        start = time #Process starts as soon as CPU is free
        finish = start + p["burst"] #Completion time = start + burst time
        waiting = start - p["arrival"] #Waiting time = start - arrival
        turnaround = finish - p["arrival"] #Turnaround = finish - arrival

        results.append({"pid": p["pid"], "start": start, "finish": finish, "waiting": waiting, "turnaround": turnaround}) #Appending scheduling outcome

        time = finish
        done.append(p) #Mark process as completed

    return results

def priority_preemptive(sample_processes):
    sample_processes = [{**p, "remaining": p["burst"]} for p in sample_processes] #Add "remaining" burst to allow preemption
    time = 0 #Current simulation time
    n = len(sample_processes) #Total number of processes
    complete = 0 #Count of finished processes
    results = {} #Store results by PID

    while complete < n: #Run until all processes complete
        available = [p for p in sample_processes if p["arrival"] <= time and p["remaining"] > 0] #Get all arrived processes with remaining CPU time

        if not available: #If nothing is ready yet, skip time until a process appears
            time += 1
            continue

        p = min(available, key=lambda x: x["priority"]) #Pick the process with the highest priority (lowest priority number)

        if p["pid"] not in results: #If this is the first time running the process, record first time it touches the CPU
            results[p["pid"]] = {"pid": p["pid"], "start": time}

        p["remaining"] -= 1 #Run for 1 time unit
        time += 1 #Move time forward

        if p["remaining"] == 0: #If process finishes now, record finish time
            results[p["pid"]]["finish"] = time
            results[p["pid"]]["waiting"] = (results[p["pid"]]["finish"] - p["arrival"] - p["burst"]) #Waiting = finish - arrival - burst
            results[p["pid"]]["turnaround"] = (results[p["pid"]]["finish"] - p["arrival"]) #Turnaround = finish - arrival
            complete += 1 #Increment completed count

    return list(results.values()) #Convert dict to list

print("Priority (Non-Preemptive):")
print_schedule(priority_non_preemptive(sample_processes))

print("\nPriority (Preemptive):")
print_schedule(priority_preemptive(sample_processes))

Priority (Non-Preemptive):
PID   Start  Finish Waiting Turnaround
P1    0      10     0       10        
P2    10     16     9       15        
P4    16     23     13      20        
P3    23     34     21      32        

Priority (Preemptive):
PID   Start  Finish Waiting Turnaround
P1    0      16     6       16        
P2    1      7      0       6         
P4    16     23     13      20        
P3    23     34     21      32        


## **Gantt Chart & Averages Code**

In [3]:
def make_gantt_chart(results):
    results = sorted(results, key=lambda x: x["start"]) #Sort tasks by start time for correct chart order

    chart = "" #For bar representation
    timeline = "" #For timestamp markers

    for r in results: #Build bars and timeline for each process
        length = r["finish"] - r["start"] #Duration of process execution
        bar = "-" * length
        chart += f"|{bar}"
        timeline += f"{r['start']}{' ' * length}" #Add start time and spacing

    timeline += f"{results[-1]['finish']}" #Add final ending time

    print("\nGantt Chart:")
    print(chart + "|")

    label_line = ""
    for r in results: #Create a line with PID labels centered in each bar
        length = r["finish"] - r["start"]
        label_line += "|" + r["pid"].center(length, "-") #Center PID inside the block
    print(label_line + "|")

    print(timeline) #Print timeline with time markers

def compute_averages(results): #Compute average waiting & turnaround times
    total_wait = sum(r["waiting"] for r in results) #Sum of all waiting times
    total_turn = sum(r["turnaround"] for r in results) #Sum of all turnaround times
    n = len(results) #Number of processes
    return {"avg_waiting": total_wait / n, "avg_turnaround": total_turn / n} # Average waiting and turnaround time

results = priority_non_preemptive(sample_processes)

print("\nPriority (Non-Preemptive):")
make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])

print("\n"+"=" * 100)

results = priority_preemptive(sample_processes)

print("\nPriority (Preemptive):")
make_gantt_chart(results)

avg = compute_averages(results)
print("\nAverage Waiting Time =", avg["avg_waiting"])
print("Average Turnaround Time =", avg["avg_turnaround"])


Priority (Non-Preemptive):

Gantt Chart:
|----------|------|-------|-----------|
|----P1----|--P2--|---P4--|-----P3----|
0          10      16       23           34

Average Waiting Time = 10.75
Average Turnaround Time = 19.25


Priority (Preemptive):

Gantt Chart:
|----------------|------|-------|-----------|
|-------P1-------|--P2--|---P4--|-----P3----|
0                1      16       23           34

Average Waiting Time = 10.0
Average Turnaround Time = 18.5
